# Weak-lensing galaxy shape and multi-band catalogue validation

## Matching statistics

> **_NOTE:_** Before running this notebook, set kernel to `main_set.ipynb'

In [2]:
import os

import astropy.coordinates as coords

from uncertainties import ufloat
import pandas as pd

from fast_histogram import histogram2d
import colorcet as cc
import matplotlib.colors as colors

from cs_util.plots import plot_histograms

from sp_validation.plots import *

## Number of matches

In [ ]:
fname = 'matched_total.txt'
if os.path.exists(fname):
    dat = np.loadtxt(fname)

    ratio = dat[:,0] / dat[:,1]
    
    xs = [ratio]
    labels = ['all']
    title = 'Matched objects ShapePipe - Gwyn'
    x_label = 'fraction of matches per tile'
    y_label = 'frequency'
    x_range = (0, 1)
    n_bin = 100
    out_path = f'{plot_dir}/hist_matched_tile_fraction.pdf'

    plot_histograms(
        xs,
        labels,
        title,
        x_label,
        y_label,
        x_range,
        n_bin,
        out_path,
        density=False,
    )
    
    xs = [dat[:,0], dat[:,1]]
    labels = ['matches', 'all SP detections']
    title = 'Matched objects ShapePipe - Gwyn'
    x_label = 'objects per tile'
    y_label = 'frequency'
    x_range = (0, 50000)
    n_bin = 100
    out_path = f'{plot_dir}/hist_matched_tile_numbers.pdf'

    plot_histograms(
        xs,
        labels,
        title,
        x_label,
        y_label,
        x_range,
        n_bin,
        out_path,
        weights=None,
        colors=None,
        linestyles=None,
        density=False,
    )

## Compute distances between matched objects

### All objects

In [ ]:
ra_sp = coords.Angle(dd['XWIN_WORLD'], unit='deg')
dec_sp = coords.Angle(dd['YWIN_WORLD'], unit='deg')
ra_ext = coords.Angle(dd['RA'], unit='deg')
dec_ext = coords.Angle(dd['Dec'], unit='deg')

In [ ]:
# Haversine distance, better than standard formula for small distances,
# avoiding cosine calls
# https://en.wikipedia.org/wiki/Great-circle_distance
delta_ra_half = (ra_sp - ra_ext) / 2
delta_dec_half = (dec_sp - dec_ext) / 2
sum_dec_half = (dec_sp + dec_ext) / 2
tmp = np.sin(delta_dec_half)**2 + ( 1 - np.sin(delta_dec_half)**2  - np.sin(sum_dec_half)**2 ) * np.sin(delta_ra_half)**2
dist_hav = 2 * np.arcsin( np.sqrt(tmp) ).to('arcsec').value

### Galaxies

In [ ]:
# Mask for valid magnitude
mval = (np.abs(dd['MAG_AUTO']) < 99)

# Combine ngmix galaxy mask with valid magnitudes
mask_match = m_gal['ngmix'] & mval

In [ ]:
ra_sp_gal = coords.Angle(dd['XWIN_WORLD'][mask_match], unit='deg')
dec_sp_gal = coords.Angle(dd['YWIN_WORLD'][mask_match], unit='deg')
ra_ext_gal = coords.Angle(dd['RA'][mask_match], unit='deg')
dec_ext_gal = coords.Angle(dd['Dec'][mask_match], unit='deg')

In [ ]:
delta_ra_half_gal = (ra_sp_gal - ra_ext_gal) / 2
delta_dec_half_gal = (dec_sp_gal - dec_ext_gal) / 2
sum_dec_half_gal = (dec_sp_gal + dec_ext_gal) / 2
tmp = np.sin(delta_dec_half_gal)**2 + ( 1 - np.sin(delta_dec_half_gal)**2  - np.sin(sum_dec_half_gal)**2 ) * np.sin(delta_ra_half_gal)**2
dist_hav_gal = 2 * np.arcsin( np.sqrt(tmp) ).to('arcsec').value

### Plots

In [ ]:
# Plot histogram of distances
xs = [dist_hav, dist_hav_gal]
labels = ['all', 'SP gal + $|r|$ < 99']
title = 'Matched objects ShapePipe - Gwyn'
x_label = 'distance [arcsec]'
y_label = 'frequency'
x_range = (0, 0.32)
n_bin = 400
out_path = f'{plot_dir}/hist_matched_dist.pdf'

plot_histograms(
    xs,
    labels,
    title,
    x_label,
    y_label,
    x_range,
    n_bin,
    out_path,
    weights=None,
    colors=None,
    linestyles=None,
    vline_x=None,
    vline_lab=None,
    density=False,
)

In [ ]:
xs = [dist_hav, dist_hav_gal]
labels = ['all', 'SP galaxies']
title = 'Matched objects ShapePipe - Gwyn'
x_label = 'distance [arcsec]'
y_label = 'frequency'
x_range = (0.3, 1)
n_bin = 400
out_path = f'{plot_dir}/hist_matched_dist_large.pdf'

plot_histograms(
    xs,
    labels,
    title,
    x_label,
    y_label,
    x_range,
    n_bin,
    out_path,
    weights=None,
    colors=None,
    linestyles=None,
    vline_x=None,
    vline_lab=None,
    density=False,
)